### 该文档旨在检查手工爬取数据的准确性

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.ops import nearest_points
import geopandas as gpd
from tqdm import tqdm

In [6]:
# 读取 Excel 文件
df = pd.read_excel("0530processing_data.xlsx")
# 检查字段
print(df.columns)

Index(['DENOMINAZI', 'Type', 'geometry', 'geometry_wgs84_OSM', 'Unnamed: 4'], dtype='object')


In [9]:
# 提取 lat/lon
def extract_lat_lon(s):
    if isinstance(s, str) and "lat=" in s and "lon=" in s:
        parts = s.split("&")
        lat = float(parts[0].split("=")[1])
        lon = float(parts[1].split("=")[1])
        return pd.Series([lat, lon])
    else:
        return pd.Series([None, None])  # 无效或缺失值

df[["lat", "lon"]] = df["geometry_wgs84_OSM"].apply(extract_lat_lon)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"
)
gdf_utm = gdf.to_crs(epsg=32632)
gdf_utm.to_file("processing_data/processing_for_test/processing.shp")  # 推荐格式


C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\3137768433.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_utm.to_file("processing_data/processing_for_test/processing.shp")  # 推荐格式
D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'geometry_wgs84_OSM' to 'geometry_w'
  ogr_write(
D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Unnamed: 4' to 'Unnamed_ 4'
  ogr_write(


In [10]:
gdf_processing = gpd.read_file(r"processing_data/processing_for_test/processing.shp").to_crs(epsg=32632)
gdf_buildings = gpd.read_file(r"origin_data\10_built_environment").to_crs(epsg=32632)

# 创建结果列
nearest_geoms = []

# tqdm 显示进度
for pt in tqdm(gdf_processing.geometry, desc="查找最近建筑"):
    # 找出距离最小的建筑几何
    nearest_geom = gdf_buildings.geometry.distance(pt).idxmin()
    nearest_geoms.append(gdf_buildings.loc[nearest_geom].geometry)

# 写入结果
gdf_processing["nearest_building_geom"] = nearest_geoms
gdf_processing_with_poly = gdf_processing.set_geometry("nearest_building_geom")

查找最近建筑:   0%|          | 0/58 [00:00<?, ?it/s]D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
查找最近建筑:   2%|▏         | 1/58 [00:00<00:12,  4.58it/s]D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
查找最近建筑:   3%|▎         | 2/58 [00:00<00:12,  4.57it/s]D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
查找最近建筑:   5%|▌         | 3/58 [00:00<00:11,  4.74it/s]D:\STUDY\anaconda3\envs\urban_morphology\lib\site-packages\shapely\measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
查找最近建筑:   7%|▋         | 4/58 [00:00<00:11,  4.65it/s]D:\STUDY\anaconda3\envs\ur

### 输出点和面的位置以方便查看

In [11]:
import matplotlib.pyplot as plt
import geopandas as gpd
import os

# 创建保存目录
output_dir =  "playground/Pro_Build_Debug"
os.makedirs(output_dir, exist_ok=True)

# 确保 geometry 是点
gdf_processing_with_poly = gdf_processing_with_poly.set_geometry("geometry")

# 遍历每个 index
for idx, row in gdf_processing.iterrows():
    fig, ax = plt.subplots(figsize=(6, 6))

    # 面：最近建筑几何体
    gpd.GeoSeries([row["nearest_building_geom"]]).plot(ax=ax, facecolor='lightblue', edgecolor='black', linewidth=1, alpha=0.6, label='建筑面')

    # 点：加工厂中心
    gpd.GeoSeries([row["geometry"]]).plot(ax=ax, color='red', markersize=30, label='加工厂点')

    ax.set_title(f"Index {idx}")
    ax.axis('equal')
    ax.axis('off')
    plt.legend()

    # 保存图像
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{idx}.png"), dpi=150)
    plt.close()

print(f"✅ 已保存 {len(gdf_processing)} 张图像到 {output_dir}/")

C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:25: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  plt.legend()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 21152 (\N{CJK UNIFIED IDEOGRAPH-52A0}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 24037 (\N{CJK UNIFIED IDEOGRAPH-5DE5}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 21378 (\N{CJK UNIFIED IDEOGRAPH-5382}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 28857 (\N{CJK UNIFIED IDEOGRAPH-70B9}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local

✅ 已保存 58 张图像到 playground/Pro_Build_Debug/


C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:25: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  plt.legend()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 21152 (\N{CJK UNIFIED IDEOGRAPH-52A0}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 24037 (\N{CJK UNIFIED IDEOGRAPH-5DE5}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 21378 (\N{CJK UNIFIED IDEOGRAPH-5382}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local\Temp\ipykernel_28792\1287638837.py:28: UserWarning: Glyph 28857 (\N{CJK UNIFIED IDEOGRAPH-70B9}) missing from font(s) Arial.
  plt.tight_layout()
C:\Users\Elena\AppData\Local

In [12]:
gdf_to_save = gdf_processing_with_poly[["DENOMINAZI","Type","nearest_building_geom"]]
gdf_to_save = gdf_to_save.rename(columns={
    "nearest_building_geom": "geometry"
})
gdf_to_save = gdf_to_save.set_crs("EPSG:32632")
gdf_to_save.to_file("processing_data/img/processing_red/processing_building.shp")